In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models
import torchvision.transforms as transforms
from torchvision.transforms.functional import resize
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import glob
import torchvision.io
import cv2
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score

In [2]:
# Custom dataset class for Lip Reading
class LipReadingDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.video_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dir))
        if '.DS_Store' in self.classes:
            self.classes.remove('.DS_Store')
        
        for label, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name, 'train')
            video_files = glob.glob(os.path.join(class_dir, "*.mp4"))
            self.video_paths.extend(video_files)
            self.labels.extend([label] * len(video_files))

        # Debug prints
        print(f"Found {len(self.video_paths)} videos across {len(self.classes)} classes.")
        if len(self.video_paths) == 0:
            print("No videos found. Please check the dataset directory structure and paths.")

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        label = self.labels[idx]
        frames = self.load_video_frames(video_path)

        if self.transform:
            frames = self.transform(frames)

        return frames, label

    def load_video_frames(self, video_path):
        frames = []
        cap = cv2.VideoCapture(video_path)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert frame to grayscale
            frame = Image.fromarray(frame)  # Convert numpy array to PIL Image
            frames.append(frame)
        cap.release()
        return frames

In [3]:
# Transform for video frames
class ToTensor:
    def __call__(self, frames):
        tensor = torch.stack([transforms.ToTensor()(frame) for frame in frames])  
        return tensor

In [4]:
# Path to the processed_selected_mp4_files directory
root_dir = './processed_selected_mp4_files'  # Update this path

# Dataset and DataLoader
transform = ToTensor()
train_dataset = LipReadingDataset(root_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, collate_fn=lambda x: collate_fn(x))
test_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, collate_fn=lambda x: collate_fn(x))

# Collate function to handle variable-length sequences
def collate_fn(batch):
    videos, labels = zip(*batch)
    max_len = max(len(video) for video in videos)
    padded_videos = []
    for video in videos:
        pad_size = max_len - len(video)
        # Ensure the tensor dimensions match for concatenation

        padded_video = torch.cat([video, torch.zeros((pad_size, video.shape[1], video.shape[2], video.shape[3]))], dim=0)  # Add padding for grayscale frames
        
        # Resize video to (max_len, 1, 224, 224)
        resized_video = torch.stack([resize(frame, (224, 224)) for frame in padded_video])
        padded_videos.append(resized_video)
        # padded_videos.append(padded_video)
    return torch.stack(padded_videos), torch.tensor(labels)

Found 20 videos across 2 classes.


In [5]:
class CNNFeatureExtractor(nn.Module):
    def __init__(self):
        super(CNNFeatureExtractor, self).__init__()
        self.cnn = models.resnet18(weights="DEFAULT")
        self.cnn.conv1 = nn.Conv2d(1, self.cnn.conv1.out_channels, kernel_size=self.cnn.conv1.kernel_size, 
                                   stride=self.cnn.conv1.stride, padding=self.cnn.conv1.padding, bias=False)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-2])  # Remove the classification layers

    def forward(self, x):
        batch_size, seq_length, c, h, w = x.size()
        x = x.view(batch_size * seq_length, c, h, w)
        features = self.cnn(x)
        features = features.view(batch_size, seq_length, -1)
        return features


In [6]:
class CNNLSTM(nn.Module):
    def __init__(self, cnn_model, hidden_dim, num_classes, num_layers=2):
        super(CNNLSTM, self).__init__()
        self.cnn = cnn_model
        self.lstm = nn.LSTM(input_size=512 * 7 * 7, hidden_size=hidden_dim, num_layers=num_layers, batch_first=False, bidirectional=True)
        self.fc = nn.Linear(hidden_dim*2, num_classes)

    def forward(self, x):
        with torch.no_grad():
            cnn_features = self.cnn(x)
        lstm_out, _ = self.lstm(cnn_features)
        output = self.fc(lstm_out[:, -1, :])  # Use the output from the last LSTM cell
        return output


In [7]:
def train(model, dataloader, criterion, optimizer, scheduler, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


num_classes = len(train_dataset.classes)  # Automatically get the number of classes
learning_rate = 0.001
num_epochs = 10

cnn_model = CNNFeatureExtractor().to(device)
model = CNNLSTM(cnn_model, hidden_dim=256, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


dataloader = train_loader

train(model, dataloader, criterion, optimizer, scheduler, num_epochs=num_epochs)


# Save the trained model
model_save_path = 'resnet_lstm_model.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Epoch 1/10, Loss: 0.8249804377555847
Epoch 2/10, Loss: 0.8183687329292297
Epoch 3/10, Loss: 0.6991655230522156
Epoch 4/10, Loss: 0.6558952331542969
Epoch 5/10, Loss: 0.6106643676757812
Epoch 6/10, Loss: 0.6167622804641724
Epoch 7/10, Loss: 0.5625695586204529
Epoch 8/10, Loss: 0.5704450011253357
Epoch 9/10, Loss: 0.5360334515571594
Epoch 10/10, Loss: 0.5357653498649597
Model saved to resnet_lstm_model.pth


In [8]:
def check_accuracy(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=torch.float32)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return 100 * acc

acc = check_accuracy(test_loader, model)

Got 10 / 20 correct (50.00)
